# Lab 01 - Techniques for Handling Large Datasets in Classification

In this scenario we will explore several techniques for handling large datasets in classification tasks. We will compare the performance of different approaches, including:

- processing the full dataset in parts
- sampling techniques
- summarization techniques
- quantization techniques

## 1. The Dataset

During this lab, we will use the [NYC Yellow Taxi Trip Data](https://www.kaggle.com/datasets/elemento/nyc-yellow-taxi-trip-data) from Kaggle.
In fact, this is a subset of the much larger and continuously updated dataset available from the [NYC Taxi & Limousine Commission (TLC)](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). The original dataset is evolving over time, with respect to its schema, therefore to avoid issues, we will use a preprocessed version from a Kaggle user.

Get familiar with the dataset page, read the description, check available columns, as well as their types and meaning. Download all available CSV files - use any means you prefer (Kaggle API, kagglehub package, direct download using browser, etc.).

In [ ]:
# write your code here


## 2. Investigate the Dataset

Get familiar with the dataset. Check the sizeof the files, the number of rows and columns, the actual data types, etc. You can use a tool of your choice (pandas, dask, pyspark, pyarrow, etc.) - the one you can install and run locally or an online tool it supports large files. Is it possible to load the entire dataset in memory? Even if it is possible in your case, try to think about scenarios where there is not enough memory available. Consequently, you should design your solutions so that they do not require loading the entire dataset at once.

In [ ]:
# write your code here


## 3. Preprocess the Dataset

Clean and preprocess the dataset, e.g., handle missing values, engineer new features, perform discretization, etc. You may approach this task in an iterative manner, i.e., you may implement a first version and then back to it later, if needed.

### Classification task

There is no predefined classification task for this dataset. You should define your own. Prepare your solution introducing some problem parameters so that they can be easily changed in the future. For example:

- You may want to predict whether the tip amount will be above a certain threshold `LARGE_TIP_THRESHOLD` (e.g., above 20% of the total amount or total amount with tip excluded) - investigate the distribution of the target variable with respect to different values of `LARGE_TIP_THRESHOLD`, plot it, and choose a reasonable value. A hint: the tip amount is reported only for cashless payments, so you may want to first filter the dataset to include only such records.
- You may want to predict whether the total amount will be above a certain threshold `HIGH_TOTAL_AMOUNT_THRESHOLD` - what is a reasonable value for this threshold?
- etc.

What evaluation approach will you use? How will you split the dataset into training and test sets?

## 4. Implement Different Techniques for Handling Large Datasets

### Classification Model

Implement a simple Naive Bayes classifier on your own that can be applied to large datasets - it should be able to process the dataset in parts, e.g., by processing one column or batch of rows at a time. Alternatively, you can use different model from available libraries, as long as it can be applied to large datasets. Be creative!

### Techniques for Handling Large Datasets

You can incorporate the following techniques for handling large datasets directly in your solution, or you can split the process into two steps - first, preprocess the data and save it (use `parquet` format), then use the saved file to train and evaluate the model. Prepare your solution so that you can easily change the parameters of the techniques you use and compare the results, computation time and the size of the processed data used for training the model.

- Process the full dataset in parts - process the dataset in parts, e.g., by processing one column at a time for Naive Bayes (compute the necessary distributions for each column) or by processing batches of rows and creating an ensemble of models.
- Sampling techniques - sample a subset of the dataset, e.g., using random sampling, stratified sampling, - introduce a parameter `FRAC` that defines the fraction of the dataset to be sampled.
- Summarization techniques - apply clustering/gridding/binning to group similar instances and aggregate the values within each cluster/bin. E.g., you can use location-related features (pickup and dropoff coordinates) to create bins and aggregate (you can take the mean for numerical features and the majority value "*mode*" for categorical features) the rides which have pickup and dropoff locations within the same bin. 
- Quantization techniques - reduce the number of distinct values for numerical features, e.g., by applying rounding, flooring, ceiling, or other techniques. Store values with reduced precision - it can reduce I/O time and memory usage and consequently speed up the computations. 

In [ ]:
# write your code here


In [ ]:
import pandas as pd
import numpy as np
import pathlib
import pyarrow.parquet as pq
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer, FunctionTransformer, OrdinalEncoder
import datetime
from sklearn.naive_bayes import CategoricalNB
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import balanced_accuracy_score, accuracy_score
import matplotlib.pyplot as plt

DATA_DIR = pathlib.Path(".") / ".." / ".." / "data"


In [ ]:
df = pd.read_parquet(DATA_DIR / "taxi_cash.parquet")
total_gb = df.memory_usage(deep = True).sum() / 10**9
print(f"Memory usage: {total_gb:.2f} GB")

In [ ]:
df.drop(columns=["RateCodeID", "RatecodeID"], inplace=True)
df.dropna(inplace=True)
df = df.sort_values(by="tpep_pickup_datetime").reset_index(drop=True)

In [ ]:
sample_index = df.sample(frac=0.05).index.sort_values()

In [ ]:
LARGE_TIP_THRESHOLD = 0.17

data = df.loc[sample_index].copy()
tip_amount = data["tip_amount"]
data.drop(columns=["tip_amount"], inplace=True)
target = tip_amount >= LARGE_TIP_THRESHOLD * (data["total_amount"] - tip_amount)
time = data["tpep_pickup_datetime"].dt.date


In [ ]:
float_cols = [
    col
    for col in data.select_dtypes(include=np.float64).columns
    if data[col].nunique() > 5
]
float_cols_1 = [
    "trip_distance",
    "pickup_longitude",
    "pickup_latitude",
    "dropoff_longitude",
    "dropoff_latitude",
    "fare_amount",
    "extra",
    "tolls_amount",
]
float_cols_2 = [
    "total_amount",
]

datetime_cols = [col for col in data.select_dtypes(include=np.datetime64).columns]
object_cols = [col for col in data.select_dtypes(include=object).columns]

transformer_nb = make_column_transformer(
    # (
    #     KBinsDiscretizer(
    #         n_bins=100, encode="ordinal", quantile_method="averaged_inverted_cdf"
    #     ),
    #     float_cols,
    # ),
    (
        KBinsDiscretizer(
            n_bins=5, encode="ordinal", quantile_method="averaged_inverted_cdf"
        ),
        float_cols_1,
    ),
    (
        KBinsDiscretizer(
            n_bins=100,
            encode="ordinal",
            strategy="quantile",
            # n_bins=100, encode="ordinal", quantile_method="averaged_inverted_cdf"
        ),
        float_cols_2,
    ),
    (
        FunctionTransformer(lambda x: x.apply(lambda col: col.dt.isocalendar().week)),
        datetime_cols,
    ),
    (
        OrdinalEncoder(),
        object_cols,
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
)
encoder_nb = OrdinalEncoder()

transformer_nb_2 = make_column_transformer(
    (
        KBinsDiscretizer(
            n_bins=100,
            encode="ordinal",
            strategy="quantile",
            # n_bins=100, encode="ordinal", quantile_method="averaged_inverted_cdf"
        ),
        float_cols_2,
    ),
    (
        FunctionTransformer(lambda x: x.apply(lambda col: col.dt.isocalendar().week)),
        datetime_cols,
    ),
    (
        OrdinalEncoder(),
        object_cols,
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
)
encoder_nb_2 = OrdinalEncoder()



transformer_tree = make_column_transformer(
    (
        FunctionTransformer(lambda x: x.apply(lambda col: col.dt.isocalendar().week)),
        datetime_cols,
    ),
    (
        OrdinalEncoder(),
        object_cols,
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
)


data_nb = encoder_nb.fit_transform(transformer_nb.fit_transform(data))

data_nb_2 = encoder_nb_2.fit_transform(
    transformer_nb_2.fit_transform(
        data.drop(
            columns=[
                "trip_distance",
                "pickup_longitude",
                "pickup_latitude",
                "dropoff_longitude",
                "dropoff_latitude",
                "fare_amount",
                "extra",
                "tolls_amount",
            ]
        )
    )
)

data_tree = transformer_tree.fit_transform(data)


In [ ]:
data.drop(
            columns=[
                "trip_distance",
                "pickup_longitude",
                "pickup_latitude",
                "dropoff_longitude",
                "dropoff_latitude",
                "fare_amount",
                "extra",
                "tolls_amount",
            ]
        )

In [ ]:
def assess(cl, X, y):
    mask = (time < datetime.date(2016, 3, 1)).to_numpy()
    X_train = X[mask]
    y_train = y[mask]

    mask = time > datetime.date(2016, 3, 15)
    X_test = X[mask]
    y_test = y[mask]



    cl.fit(X_train, y_train)
    print(balanced_accuracy_score(y_test, cl.predict(X_test)))
    return cl


cl = assess(DecisionTreeClassifier(min_samples_leaf=5000), data_tree, target)
cl2 = assess(CategoricalNB(), data_nb, target)
cl3 = assess(CategoricalNB(), data_nb_2, target)


In [ ]:
plt.figure(figsize=(40, 20))
plot_tree(
    cl,
    filled=True,
    rounded=True,
    fontsize=12,
)
plt.show()